# 로컬(주피터)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install webdriver_manager

## 코랩

In [ ]:
# @title
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap
# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500
Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300
Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.sWaBrO1xAm/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.qZBanlYZpc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.6hKfGjvZio/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [ ]:
pip install chromedriver_autoinstaller

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import random

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from time import sleep
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re
import requests
import os
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

# Dataframe
import pandas as pd

# 2023.08.02 셀레니움 라이브러리 수정사항 반영
# https://stackoverflow.com/questions/76428561/typeerror-webdriver-init-got-multiple-values-for-argument-options
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service, options=options)

# 잠깐!

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os
import subprocess

from datetime import datetime

def get_news_url(query, start_date, end_date, page=1):
    # 입력된 날짜를 YYYY.MM.DD 형식으로 변환
    start_date_formatted = datetime.strptime(start_date, "%Y-%m-%d").strftime("%Y.%m.%d")
    end_date_formatted = datetime.strptime(end_date, "%Y-%m-%d").strftime("%Y.%m.%d")

    base_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=0&photo=0&field=0&pd=3&cluster_rank=20&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:from{}to{},a:all&start={}'

    formatted_url = base_url.format(query, start_date_formatted, end_date_formatted, (page - 1) * 10 + 1)
    return formatted_url

In [ ]:
def crawl_naver_news(query, start_date, end_date):
    news_dict = {}
    idx = 0
    cur_page = 1

    while True:
        url = get_news_url(query, start_date, end_date, page=cur_page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126)
        table = soup.find('ul', {'class': 'list_news'})
        li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
        area_list = [li.find('div', {'class': 'news_area'}) for li in li_list]
        a_list = [area.find('a', {'class': 'news_tit'}) for area in area_list]

        if not a_list:
            break  # 뉴스 기사가 더 이상 없으면 종료

        for n in a_list:
            news_dict[idx] = {'title': n.get('title'), 'url': n.get('href')}
            idx += 1

        cur_page += 1

    return news_dict

# 빅카인즈 기업 검색

In [ ]:
com_sec_all = pd.read_csv('/content/drive/MyDrive/뉴스레터 서비스(공유)/300대 기업 최종 데이터.csv')

In [ ]:
# @title
def get_page_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    cards = soup.find_all('div', class_='card')

    page_data = []
    for card in cards:
        company_name = card.find('p', {'class': 'k-name p-ellipsis'}).text
        industry = card.find('dd', {'class': 'p-ellipsis'}).text
        page_data.append({'Company': company_name, 'Industry': industry})

    return page_data

'''
data = []
start_page=1

while True:
    # 웹 페이지 열기
    url = f"https://www.bigkinds.or.kr/v2/depthAnalysis/company.do?codeGroupId=corp_top300_list&page={start_page}"
    driver.get(url)

    # 게시글 테이블 로딩을 기다림
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-wrap.cl div.card")))

    # 현재 페이지에서 회사 정보 수집
    cards = driver.find_elements(By.CSS_SELECTOR, "div.card-wrap.cl div.card")
    for card in cards:
        company_name_kr = card.find_element(By.CSS_SELECTOR, "p.k-name.p-ellipsis").text.strip()
        company_name_en = card.find_element(By.CSS_SELECTOR, "p.e-name.p-ellipsis").text.strip()
        industry = card.find_element(By.CSS_SELECTOR, "dd.p-ellipsis").text.strip()
        representative = card.find_elements(By.TAG_NAME, "dd")[-1].text.strip()

        # 회사 정보를 딕셔너리에 저장
        company_info = {
            "한글 회사명": company_name_kr,
            "영문 회사명": company_name_en,
            "업종": industry,
            "대표이름": representative
        }

        # 딕셔너리를 리스트에 추가
        data.append(company_info)

    start_page += 1

    if start_page > 19:
      break

results = pd.DataFrame(data)
'''
# 데이터프레임 저장
#company = results
company = pd.read_csv('/content/drive/My Drive/뉴스레터 서비스(공유)/company_list_dataframe.csv')#, index=False)
company_list = list(company['한글 회사명'])
#company_list

In [ ]:
# @title
# # @title
# # '한글 회사명'을 기준으로 병합
# import pandas as pd
# #company = pd.read_csv('/content/drive/My Drive/company_list_dataframe.csv')
# vs = pd.read_excel('/content/drive/MyDrive/company_classification.xlsx')
# catch = pd.read_csv('/content/drive/MyDrive/catch_company')
# company_list = list(company['한글 회사명'])

# merged_df = pd.merge(catch, company, left_on='이름', right_on='한글 회사명', how='right')
# df = merged_df.drop(['이름', '영문 회사명'], axis=1)
# df = df.rename(columns={'업종_x':'업종'})
# # '업종' 열의 NaN 값을 '업종_y' 열의 값으로 대체
# df['업종'] = df['업종'].fillna(df['업종_y'])
# df = df.drop(['업종_y'], axis=1)
# columns = list(df.columns)
# new_order = columns[-2:] + columns[:-2]
# df = df[new_order]

# # 'BEFORE'와 'AFTER' 열을 사용하여 매핑 생성
# mapping = dict(zip(vs['BEFORE'], vs['AFTER']))
# df['업종'] = df['업종'].map(mapping).fillna(df['업종'])
# # '업종' 열에서 '서비스업'을 포함하는 인덱스의 값을 '고객 서비스·리테일'로 변경
# df.loc[df['업종'].str.contains('서비스업', na=False), '업종'] = '고객 서비스·리테일'
# df.loc[df['업종'].str.contains('금융', na=False), '업종'] = '금융'
# df.loc[df['업종'].str.contains('증권', na=False), '업종'] = '금융'
# df.loc[df['업종'].str.contains('보험', na=False), '업종'] = '금융'
# df.loc[df['업종'].str.contains('은행', na=False), '업종'] = '금융'
# df.loc[df['업종'].str.contains('화학', na=False), '업종'] = '에너지·화학'
# df.loc[df['업종'].str.contains('발전', na=False), '업종'] = '에너지·화학'
# df.loc[df['업종'].str.contains('처리', na=False), '업종'] = '에너지·화학'
# df.loc[df['업종'].str.contains('자동차', na=False), '업종'] = '자동차·운송'
# df.loc[df['업종'].str.contains('운송', na=False), '업종'] = '자동차·운송'
# df.loc[df['업종'].str.contains('선박', na=False), '업종'] = '자동차·운송'
# df.loc[df['업종'].str.contains('부동산', na=False), '업종'] = '지주회사'
# df.loc[df['업종'].str.contains('임대업', na=False), '업종'] = '지주회사'
# df.loc[df['업종'].str.contains('항공', na=False), '업종'] = '호텔·여행·항공'
# df.loc[df['업종'].str.contains('호텔', na=False), '업종'] = '호텔·여행·항공'
# df.loc[df['업종'].str.contains('옷', na=False), '업종'] = '섬유·의류'
# df.loc[df['업종'].str.contains('의류', na=False), '업종'] = '섬유·의류'
# df.loc[df['업종'].str.contains('의료', na=False), '업종'] = '의료·제약·바이오'
# df.loc[df['업종'].str.contains('제약', na=False), '업종'] = '의료·제약·바이오'
# df.loc[df['업종'].str.contains('통신', na=False), '업종'] = '네트워크·통신'
# df.loc[df['업종'].str.contains('게임', na=False), '업종'] = '게임'
# df.loc[df['업종'].str.contains('음료', na=False), '업종'] = '식·음료'
# df.loc[df['업종'].str.contains('소주', na=False), '업종'] = '식·음료'
# df.loc[df['업종'].str.contains('빵', na=False), '업종'] = '식·음료'
# df.loc[df['업종'].str.contains('화장품', na=False), '업종'] = '생활용품·화장품'
# df.loc[df['업종'].str.contains('건설', na=False), '업종'] = '건설·토목'
# df.loc[df['업종'].str.contains('컴퓨터', na=False), '업종'] = 'IT·플랫폼'
# df.loc[df['업종'].str.contains('조절', na=False), '업종'] = '기계장비'
# df.loc[df['업종'].str.contains('소매업', na=False), '업종'] = '고객 서비스·리테일'
# df.loc[df['업종'].str.contains('중개업', na=False), '업종'] = '고객 서비스·리테일'

# unique_jobs_df = set(df['업종'].unique())
# after_jobs_vs = set(vs['AFTER'])
# overlapping = unique_jobs_df.intersection(after_jobs_vs)
# only_in_df = unique_jobs_df.difference(after_jobs_vs)

# # '업종' 열에서 only_in_df에 속하고 '제조업'을 포함하는 값을 '제조업 기타'로 변경
# df['업종'] = df['업종'].apply(lambda x: '제조업 기타' if x in only_in_df and '제조업' in x else x)

# df.to_csv('/content/drive/MyDrive/modified_company.csv', index=False)

# df1 = df[df['업종']=='전기·전자']
# df2 = df[df['업종']=='자동차·운송']
# df3 = df[df['업종']=='공공·복지']
# df4 = df[df['업종']=='에너지·화학']
# df5 = df[df['업종']=='금융']
# df6 = df[df['업종']=='고객 서비스·리테일']
# df7 = df[df['업종']=='반도체·디스플레이']
# df8 = df[df['업종']=='무역']
# df9 = df[df['업종']=='제조업 기타']
# df10 = df[df['업종']=='식·음료']
# df11 = df[df['업종']=='전기·전자']
# df12 = df[df['업종']=='철강·금속']
# df13 = df[df['업종']=='네트워크·통신']
# df14 = df[df['업종']=='건설·토목']
# df15 = df[df['업종']=='지주회사']
# df16 = df[df['업종']=='IT·플랫폼']
# df17 = df[df['업종']=='기계장비']
# df18 = df[df['업종']=='호텔·여행·항공']
# df19 = df[df['업종']=='조선']
# df20 = df[df['업종']=='생활용품·화장품']
# df21 = df[df['업종']=='미디어·문화']
# df22 = df[df['업종']=='광고·홍보']
# df23 = df[df['업종']=='섬유·의류']
# df24 = df[df['업종']=='제조업']
# df25 = df[df['업종']=='솔루션·SI·CRM·ERP']
# df26 = df[df['업종']=='의료·제약·바이오']
# df27 = df[df['업종']=='게임']
# df28 = df[df['업종']=='목재·제지']

# 빅카인즈X 네이버 기사 가져오기

In [ ]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, progressbar, json, glob, time
from tqdm import tqdm

# 모든 페이지의 기사를 저장할 리스트
all_articles = []

2# 네이버 검색 API 호출 URL
api_url = "https://openapi.naver.com/v1/search/news.json"

client_id = 'Zwdz_lbupo91QAvrtqsz'
client_secret = 'ifGZTCfREm'

# API 호출 시 사용할 헤더 정보 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 날짜 변환 함수 수정
def convert_to_date(date_str):
    # 이미 datetime 객체인 경우, 날짜 부분만 반환
    if isinstance(date_str, datetime):
        return date_str.date()

    # 문자열인 경우 날짜 변환 시도
    if isinstance(date_str, str):
        for fmt in ('%Y-%m-%d', '%Y.%m.%d.'):
            try:
                return datetime.strptime(date_str, fmt).date()
            except ValueError:
                pass

    # 변환할 수 없는 경우 원래 값을 반환
    return date_str

# 상대적 날짜를 절대적 날짜로 변환하는 함수
def get_absolute_date(relative_date_str):
    numbers = re.findall(r'\d+', relative_date_str)
    days_ago = int(numbers[0]) if numbers else 0
    absolute_date = datetime.now() - timedelta(days=days_ago)
    return absolute_date.strftime('%Y-%m-%d')

def get_absolute_date_all(relative_date_str):
    now = datetime.now()

    # 숫자를 추출합니다.
    numbers = re.findall(r'\d+', relative_date_str)

    # 숫자를 추출하지 못한 경우, 0으로 설정합니다.
    if not numbers:
        return now.strftime('%Y-%m-%d %H:%M:%S')

    # 상대적인 시간의 단위를 추출합니다.
    unit = relative_date_str.split(numbers[0])[-1].strip()

    # 시간, 일, 분에 따라 다른 계산을 수행합니다.
    if unit == '시간 전':
        absolute_date = now - timedelta(hours=int(numbers[0]))
    elif unit == '일 전':
        absolute_date = now - timedelta(days=int(numbers[0]))
    elif unit == '분 전':
        absolute_date = now - timedelta(minutes=int(numbers[0]))
    else:
        raise ValueError("Unsupported relative date format")

    return absolute_date.strftime('%Y-%m-%d %H:%M:%S')

def parse_date(date):
    # 입력값을 문자열로 변환하여 앞에서부터 4자리는 year, 4자리는 month, 나머지는 date로 추출
    date_str = str(date)
    year = date_str[:4]
    month = date_str[4:6]
    day = date_str[6:]
    r =  f'{year}.{month}.{day}.'
    # .으로 구분하여 반환
    return str(r)

# 300대 기업 하루치 크롤링

### 12 hour 마다 업데이트

In [ ]:
!pip install schedule

아래 숨긴건 selenium

In [ ]:
# @title
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from time import sleep
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re
import requests
import os
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

# Dataframe
import pandas as pd

# 2023.08.02 셀레니움 라이브러리 수정사항 반영
# https://stackoverflow.com/questions/76428561/typeerror-webdriver-init-got-multiple-values-for-argument-options
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
# @title
# 날짜 포맷 설정
def get_formatted_date():
    now = datetime.now()
    period = "AM" if now.hour < 12 else "PM"
    formatted_date = now.strftime('%Y.%m.%d.%I.%M') + period
    return formatted_date

def get_absolute_date(relative_date_str):
    if '일 전' in relative_date_str:
        days_ago = int(relative_date_str.split('일 전')[0])
        absolute_date = datetime.now() - timedelta(days=days_ago)
        return absolute_date.strftime('%Y-%m-%d')
    elif '시간 전' in relative_date_str:
        hours_ago = int(relative_date_str.split('시간 전')[0])
        absolute_date = datetime.now() - timedelta(hours=hours_ago)
        return absolute_date.strftime('%Y-%m-%d')
    elif '분 전' in relative_date_str:
        minutes_ago = int(relative_date_str.split('분 전')[0])
        absolute_date = datetime.now() - timedelta(minutes=minutes_ago)
        return absolute_date.strftime('%Y-%m-%d')
    return relative_date_str

def create_folder():
    now = datetime.now()
    year = now.strftime('%Y')
    month = now.strftime('%m')
    folder_name = f'/content/drive/MyDrive/뉴스레터 서비스(공유)/{year}_{month}'

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"폴더가 생성되었습니다: {folder_name}")
    else:
        print(f"폴더가 이미 존재합니다: {folder_name}")

    return folder_name

def delete_files_with_prefix(folder_path, prefix):
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            if filename.startswith(prefix):
                file_path = os.path.join(folder_path, filename)
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    print(f"파일이 삭제되었습니다: {file_path}")
                else:
                    print(f"파일이 아닙니다: {file_path}")
    else:
        print(f"폴더가 존재하지 않습니다: {folder_path}")

In [ ]:
# 시작 날짜는 하루전
start_date = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')
# 끝나는 날짜는 현재 시간
end_date = datetime.now().strftime('%Y%m%d')

start_time = get_formatted_date()

start_date_ = datetime.strptime(start_date, '%Y%m%d')
ed = datetime.strptime(end_date, '%Y%m%d')
ed += timedelta(days=1)  # ed에 1일을 더함
ed_str = ed.strftime('%Y%m%d')
news_list = []

In [ ]:
for company in company_list:
    query = company
    query = query.replace(' ', '+')
    print(f'    {query} 뉴스 기사 검색 중')

    news_url_ = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={query}&sort=2'
    search_option = '&nso=so%3Ar%2Cp%3Afrom{}to{}'.format(start_date.replace('.', ''), ed_str.replace('.', ''))
    news_url_ = news_url_ + search_option
    news_url_= str(news_url_)

    start_page = 1
    results_per_page = 10

    i = 1

    while True:
        start_value = i * results_per_page + 1
        news_url = news_url_ + "&start=" + str(start_value)
        #print("news url: ", news_url)
        driver.get(news_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 페이지에 기사가 있는지 확인
        articles_exist = False
        for li in soup.find_all('li', {'id': re.compile('sp_nws.*')}):
            articles_exist = True
            title_tag = li.find('a', class_='news_tit')
            title = title_tag.get_text().strip() if title_tag else '제목 없음'
            url = title_tag['href'] if title_tag else 'URL 없음'
            press_tag = li.find('a', class_='info press')
            press = press_tag.get_text().strip() if press_tag else '언론사 없음'

            date_tag = li.find('span', class_='info')
            date = date_tag.get_text()
            if '전' in date:
                date = get_absolute_date(date)  # 수정된 부분: get_absolute_date 함수 사용

            # 기사 게시일이 종료 날짜보다 이전인 경우에만 크롤링을 진행
            try:
                article_date = datetime.strptime(date, '%Y-%m-%d')
            except ValueError:
                # 날짜 형식이 아닌 경우에는 기사를 건너뜁니다.
                #print("    날짜 형식이 아닌 데이터입니다. 기사를 건너뜁니다.")
                continue

            if article_date < ed:  # 수정된 부분: start_date_와 end_date_ 변수 수정
                #print('        기업: ', query)
                #print('        언론사: ', press)
                #print('        제목: ', title)
                #print('        일자: ', date)
                #print('        URL: ', url)
                news_dict = {'기업': query,'언론사': press, '제목': title, '일자': date, 'URL': url}
                news_list.append(news_dict)
                print(f'       {query} 기업 {i}번째 -> 언론사: {press}, 제목: {title}, 일자: {date}, URL: {url}')

        # 페이지에 기사가 없으면 루프를 종료합니다.
        if not articles_exist:
            print("        더 이상 기사가 존재하지 않습니다. 크롤링을 종료합니다.")
            break

        i += 1
    print('    ', query, ' 기업의 뉴스 크롤링 완료 ***')

숨긴 코드는 건들지 말 것. update 느려서 숨김

In [ ]:
# @title
import schedule
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
import concurrent.futures

## 2023.08.02 셀레니움 라이브러리 수정사항 반영
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')

# 날짜 포맷 설정
def get_formatted_date():
    now = datetime.now()
    period = "AM" if now.hour < 12 else "PM"
    formatted_date = now.strftime('%Y.%m.%d.%I.%M') + period
    return formatted_date

def get_absolute_date(relative_date_str):
    if '일 전' in relative_date_str:
        days_ago = int(relative_date_str.split('일 전')[0])
        absolute_date = datetime.now() - timedelta(days=days_ago)
        return absolute_date.strftime('%Y-%m-%d')
    elif '시간 전' in relative_date_str:
        hours_ago = int(relative_date_str.split('시간 전')[0])
        absolute_date = datetime.now() - timedelta(hours=hours_ago)
        return absolute_date.strftime('%Y-%m-%d')
    elif '분 전' in relative_date_str:
        minutes_ago = int(relative_date_str.split('분 전')[0])
        absolute_date = datetime.now() - timedelta(minutes=minutes_ago)
        return absolute_date.strftime('%Y-%m-%d')
    return relative_date_str

def create_folder():
    now = datetime.now()
    year = now.strftime('%Y')
    month = now.strftime('%m')
    folder_name = f'/content/drive/MyDrive/뉴스레터 서비스(공유)/{year}_{month}'

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"폴더가 생성되었습니다: {folder_name}")
    else:
        print(f"폴더가 이미 존재합니다: {folder_name}")

    return folder_name

def delete_files_with_prefix(folder_path, prefix):
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            if filename.startswith(prefix):
                file_path = os.path.join(folder_path, filename)
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    print(f"파일이 삭제되었습니다: {file_path}")
                else:
                    print(f"파일이 아닙니다: {file_path}")
    else:
        print(f"폴더가 존재하지 않습니다: {folder_path}")

def crawl_news(company_list_path):
    # 시작 날짜는 현재 시간에서 1일 뺀 날짜
    start_date = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')
    # 끝나는 날짜는 현재 시간
    end_date = datetime.now().strftime('%Y%m%d')

    # 시작 시간 포맷 설정
    start_time = get_formatted_date()

    print(f'지금 시간은 {start_time}')

    start_date_ = datetime.strptime(start_date, '%Y%m%d')
    ed = datetime.strptime(end_date, '%Y%m%d')
    ed += timedelta(days=1)  # ed에 1일을 더함
    ed_str = ed.strftime('%Y%m%d')

    company = pd.read_csv(company_list_path)
    company_list = list(company['한글 회사명'])

    news_list = []

    def process_company(company):
        query = company
        query = query.replace(' ', '+')
        print(f'  {query} 뉴스 기사 검색 중')

        news_url_ = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={query}&sort=2'
        search_option = '&nso=so%3Ar%2Cp%3Afrom{}to{}'.format(start_date.replace('.', ''), ed_str.replace('.', ''))
        news_url_ = news_url_ + search_option
        news_url_ = str(news_url_)

        results_per_page = 10
        i = 0

        driver = webdriver.Chrome(service=service, options=options)

        while True:
            start_value = i * results_per_page + 1
            news_url = news_url_ + "&start=" + str(start_value)
            print("news url: ", news_url)
            driver.get(news_url)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # 페이지에 기사가 있는지 확인
            articles_exist = False
            for li in soup.find_all('li', {'id': re.compile('sp_nws.*')}):
                articles_exist = True
                title_tag = li.find('a', class_='news_tit')
                title = title_tag.get_text().strip() if title_tag else '제목 없음'
                url = title_tag['href'] if title_tag else 'URL 없음'
                press_tag = li.find('a', class_='info press')
                press = press_tag.get_text().strip() if press_tag else '언론사 없음'

                date_tag = li.find('span', class_='info')
                date = date_tag.get_text()
                if '전' in date:
                    date = get_absolute_date(date)

                # 기사 게시일이 종료 날짜보다 이전인 경우에만 크롤링을 진행
                try:
                    article_date = datetime.strptime(date, '%Y-%m-%d')
                except ValueError:
                    # 날짜 형식이 아닌 경우에는 기사를 건너뜁니다.
                    continue

                if article_date < ed:
                    news_dict = {'기업': query, '언론사': press, '제목': title, '일자': date, 'URL': url}
                    news_list.append(news_dict)

            # 페이지에 기사가 없으면 루프를 종료합니다.
            if not articles_exist:
                print("  더 이상 기사가 존재하지 않습니다. 크롤링을 종료합니다.")
                break

            i += 1
        driver.quit()
        print(f'  {query} 기업의 뉴스 크롤링 완료 ***')

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(process_company, company_list)

    # 수집된 뉴스 리스트를 파일로 저장하거나 DB에 저장
    folder_path = create_folder()
    save_news_to_csv(news_list, start_time, folder_path)

def save_news_to_csv(news_list, start_time, folder_path):
    current_time = get_formatted_date()
    df = pd.DataFrame(news_list)
    file_path = os.path.join(folder_path, f'news_{start_time}_to_{current_time}.csv')
    df.to_csv(file_path, index=False, encoding='utf-8-sig')

def schedule_crawling(company_list_path):
    schedule.every().day.at("00:00").do(crawl_news, company_list_path)

    while True:
        schedule.run_pending()
        time.sleep(1)

In [ ]:
#company_list_path = input('기업 리스트 CSV 파일 경로를 입력하세요 (예: /path/to/company_list.csv): ')
company_list_path = '/content/drive/My Drive/뉴스레터 서비스(공유)/company_list_dataframe.csv'
#schedule_crawling(company_list_path)

In [ ]:
# @title
# 뉴스 크롤링
def crawl_news(start_date, end_date, company_list_path):
    # 날짜 입력 받기
    start_time = get_formatted_date()

    start_date_ = datetime.strptime(start_date, '%Y%m%d')
    ed = datetime.strptime(end_date, '%Y%m%d')
    ed += timedelta(days=1)                             # ed에 1일을 더함
    ed_str = ed.strftime('%Y%m%d')

    company = pd.read_csv(company_list_path)#, index=False)
    company_list = list(company['한글 회사명'])

    news_list = []

    for company in company_list:
        query = company
        query = query.replace(' ', '+')
        print(query + ' 뉴스 기사 검색 중')

        news_url_ = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={query}&sort=2'
        search_option = '&nso=so%3Ar%2Cp%3Afrom{}to{}'.format(start_date.replace('.', ''), ed_str.replace('.', ''))
        news_url_ = news_url_ + search_option
        news_url_ = str(news_url_)

        start_page = 1
        results_per_page = 10

        i = 0

        while True:
            start_value = i * results_per_page + 1
            news_url = news_url_ + "&start=" + str(start_value)
            print("news url: ", news_url)
            driver.get(news_url)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # 페이지에 기사가 있는지 확인
            articles_exist = False
            for li in soup.find_all('li', {'id': re.compile('sp_nws.*')}):
                articles_exist = True
                title_tag = li.find('a', class_='news_tit')
                title = title_tag.get_text().strip() if title_tag else '제목 없음'
                url = title_tag['href'] if title_tag else 'URL 없음'
                press_tag = li.find('a', class_='info press')
                press = press_tag.get_text().strip() if press_tag else '언론사 없음'

                date_tag = li.find('span', class_='info')
                date = date_tag.get_text()
                if '전' in date:
                    date = get_absolute_date(date)  # 수정된 부분: get_absolute_date 함수 사용

                # 기사 게시일이 종료 날짜보다 이전인 경우에만 크롤링을 진행
                try:
                    article_date = datetime.strptime(date, '%Y-%m-%d')
                except ValueError:
                    # 날짜 형식이 아닌 경우에는 기사를 건너뜁니다.
                    continue

                if article_date < ed:  # 수정된 부분: start_date_와 end_date_ 변수 수정
                    news_dict = {'기업': query, '언론사': press, '제목': title, '일자': date, 'URL': url}
                    news_list.append(news_dict)

            # 페이지에 기사가 없으면 루프를 종료합니다.
            if not articles_exist:
                print("더 이상 기사가 존재하지 않습니다. 크롤링을 종료합니다.")
                break

            i += 1
        print(query, ' 기업의 뉴스 크롤링 완료 ***')

    # 수집된 뉴스 리스트를 파일로 저장하거나 DB에 저장
    folder_path = create_folder()
    save_news_to_csv(news_list, start_time, folder_path)

In [ ]:
'''
# @title
for company in company_list:
    query = company
    query = query.replace(' ', '+')
    print(query+' 뉴스 기사 검색 중')

    news_url_ = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={query}&sort=2'
    search_option = '&nso=so%3Ar%2Cp%3Afrom{}to{}'.format(start_date.replace('.', ''), ed_str.replace('.', ''))
    news_url_ = news_url_ + search_option
    news_url_= str(news_url_)

    start_page = 1
    results_per_page = 10

    i = 0

    while True:
        start_value = i * results_per_page + 1
        news_url = news_url_ + "&start=" + str(start_value)
        print("news url: ", news_url)
        driver.get(news_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 페이지에 기사가 있는지 확인
        articles_exist = False
        for li in soup.find_all('li', {'id': re.compile('sp_nws.*')}):
            articles_exist = True
            title_tag = li.find('a', class_='news_tit')
            title = title_tag.get_text().strip() if title_tag else '제목 없음'
            url = title_tag['href'] if title_tag else 'URL 없음'
            press_tag = li.find('a', class_='info press')
            press = press_tag.get_text().strip() if press_tag else '언론사 없음'

            date_tag = li.find('span', class_='info')
            date = date_tag.get_text()
            if '전' in date:
                date = get_absolute_date(date)  # 수정된 부분: get_absolute_date 함수 사용

            # 기사 게시일이 종료 날짜보다 이전인 경우에만 크롤링을 진행
            try:
                article_date = datetime.strptime(date, '%Y-%m-%d')
            except ValueError:
                # 날짜 형식이 아닌 경우에는 기사를 건너뜁니다.
                #print("    날짜 형식이 아닌 데이터입니다. 기사를 건너뜁니다.")
                continue

            if article_date < ed:  # 수정된 부분: start_date_와 end_date_ 변수 수정
                #print('        기업: ', query)
                #print('        언론사: ', press)
                #print('        제목: ', title)
                #print('        일자: ', date)
                #print('        URL: ', url)
                news_dict = {'기업': query,'언론사': press, '제목': title, '일자': date, 'URL': url}
                news_list.append(news_dict)

        # 페이지에 기사가 없으면 루프를 종료합니다.
        if not articles_exist:
            print("        더 이상 기사가 존재하지 않습니다. 크롤링을 종료합니다.")
            break

        i += 1
    print('    ', query, ' 기업의 뉴스 크롤링 완료 ***')
'''

In [ ]:
#ALL = pd.DataFrame(news_list)
ALL.to_csv(f'/content/drive/My Drive/뉴스레터 서비스(공유)/{start_date}_{end_date}_기사크롤링.csv', index=False)

In [ ]:
news_all = pd.read_csv(f'/content/drive/My Drive/뉴스레터 서비스(공유)/{start_date}_{end_date}_기사크롤링.csv')
ALL = pd.DataFrame(news_all)
ALL.tail(1)

,기업,언론사,제목,일자,URL,업종,규모
12474,오리온,이코노뉴스,"홈플러스, 'AI 가격혁명·서머 싹쓸이' 할인전 진행…인기 먹거리 '반값'",2024-06-27,http://www.econonews.co.kr/news/articleView.ht...,식·음료,중견기업


In [ ]:
# ALL 데이터프레임과 com_sec_all 데이터프레임을 기업 이름을 기준으로 병합
merged_df = pd.merge(ALL, com_sec_all[['기업이름', '업종', '규모']], left_on='기업', right_on='기업이름', how='left')
# 불필요한 열 삭제 (기업이름 열은 기준 열로 사용했으므로 삭제)
columns_to_remove = ['Unnamed: 0', '업종_x', '규모_x']
merged_df.drop(columns=columns_to_remove, inplace=True, errors='ignore')
merged_df.rename(columns={'업종_y': '업종', '규모_y': '규모'}, inplace=True)
# 필요한 열만 남기고 중복 열 제거 및 이름 변경
merged_df.drop(columns=['기업이름'], inplace=True)
ALL = merged_df

# CSV 파일로 저장
path = f'/content/drive/My Drive/뉴스레터 서비스(공유)/{start_date}_{end_date}_기사크롤링.csv'
if not os.path.exists(path):
        os.makedirs(path)
ALL.to_csv(path, index=False)
ALL

,기업,언론사,제목,일자,URL,업종,규모
0,삼성전자,아주경제,아리셀 화재로 번진 '리튬 공포증'...韓 대응 매뉴얼이 없다,2024-06-26,https://www.ajunews.com/view/20240625154036606,전기·전자,대기업
1,삼성전자,AP신문,"이재용 'SW 씨앗' 철학에…SSAFY, 고졸까지 확대된다",2024-06-26,https://www.apnews.kr/news/articleView.html?id...,전기·전자,대기업
2,삼성전자,SBS Biz,"민주당 ""반도체에 100조 지원""…반도체 특별법 발의",2024-06-26,https://biz.sbs.co.kr/article_hub/20000178199?...,전기·전자,대기업
3,삼성전자,글로벌이코노믹,"[모닝픽] SK온, 엑손모빌과 손잡고 미국산 리튬 확보",2024-06-26,http://www.g-enews.com/ko-kr/news/article/news...,전기·전자,대기업
4,삼성전자,연합뉴스,"삼성전자 DS부문, 오늘 글로벌전략회의…반도체 돌파구 모색 총력",2024-06-26,https://www.yna.co.kr/view/AKR2024062515670000...,전기·전자,대기업
...,...,...,...,...,...,...,...
12470,오리온,경북신문,"구미시, 창업지원기관 협의회 개최...지원 정책 연계 강화",2024-06-27,http://www.kbsm.net/news/view.php?idx=438336,식·음료,중견기업
12471,오리온,경북신문,"KBL 컵대회, 10월 5일∼13일 제천서 개최…상무 포함 11개 팀 출전",2024-06-27,http://www.kbsm.net/news/view.php?idx=438361,식·음료,중견기업
12472,오리온,업다운뉴스,"파격 변신의 성과...오리온, '초코파이 하우스' 출시 넉달새 2400만개 판매",2024-06-27,http://www.updownnews.co.kr/news/articleView.h...,식·음료,중견기업
12473,오리온,머니투데이,출시 4개월 2400만개 팔려…초코파이에 '크림' 넣은 오리온의 속셈,2024-06-27,http://news.mt.co.kr/mtview.php?no=20240627140...,식·음료,중견기업


### sample.csv 파일 전처리

In [ ]:
sample_all = ALL.copy()

In [ ]:
# @title
# '게시일' 열을 날짜 형식으로 변환
sample_all['일자'] = sample_all['일자'].apply(convert_to_date)

# 날짜 데이터만을 포함하는 데이터프레임 정렬
# '게시일' 열에서 datetime.date 타입만을 선택
sample_all = sample_all[pd.to_datetime(sample_all['일자'], errors='coerce').notna()]
sample_all = sample_all.sort_values(by='일자', ascending=False)
sample_all = sample_all.reset_index(drop=True)
sample_filtered_all = sample_all.copy()

# 특정 언론사 리스트 정의
allowed_presses = ['연합뉴스', '동아일보', '중앙일보', '한겨레', '한국경제']

# 먼저 '언론사' 열이 allowed_presses에 속하는 행만 남기고, 나머지 행 삭제
sample_filtered_all = sample_all[sample_all['언론사'].isin(allowed_presses)]
sample_filtered_all['URL'] = sample_filtered_all .apply(lambda row: row['URL'].split('?')[0] if row['언론사'] == '연합뉴스' else row['URL'], axis=1)

# 정규 표현식을 사용하여 'URL' 열에서 'PYH' 또는 'GYH'로 시작하는지 확인하는 패턴
pattern = re.compile(r'^https://www.yna.co.kr.*(PYH|GYH)')

# 조건에 따라 행을 제거
sample_filtered_all = sample_filtered_all[~((sample_filtered_all['언론사'] == '연합뉴스') & (sample_filtered_all['URL'].str.contains(pattern)))]
sample_filtered_all = sample_filtered_all[~((sample_filtered_all['언론사'] == '한국경제') & (sample_filtered_all['URL'].str.endswith('r')))]

# 이후 '제목' 열에서 중복 제거, 첫 번째 등장하는 항목만 유지
sample_filtered_all = sample_filtered_all.drop_duplicates(subset=['제목'], keep='first')
sample_filtered_all = sample_filtered_all.reset_index(drop=True)
sample_filtered_all['일자'] = pd.to_datetime(sample_filtered_all['일자'])
sample_filtered_all= sample_filtered_all[~sample_filtered_all['URL'].str.startswith('https://en.yna.co.kr')]

no_content_all = sample_filtered_all.copy()
no_content_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/{start_date}_{end_date}_본문없음_최종_전체기업_크롤링.csv')

<ipython-input-511-67a5e569df84>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_filtered_all['URL'] = sample_filtered_all .apply(lambda row: row['URL'].split('?')[0] if row['언론사'] == '연합뉴스' else row['URL'], axis=1)
<ipython-input-511-67a5e569df84>:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sample_filtered_all = sample_filtered_all[~((sample_filtered_all['언론사'] == '연합뉴스') & (sample_filtered_all['URL'].str.contains(pattern)))]


# 전체 기업 크롤링(본문 제외) 데이터

In [ ]:
# @title
yna_all = sample_filtered_all[sample_filtered_all['언론사']=='연합뉴스']
yna_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/연합_{start_date}_{end_date}_본문없음_최종_전체기업_크롤링.csv')

donga_all = sample_filtered_all[sample_filtered_all['언론사']=='동아일보']
donga_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/동아_{start_date}_{end_date}_본문없음_최종_전체기업_크롤링.csv')

joong_all = sample_filtered_all[sample_filtered_all['언론사']=='중앙일보']
joong_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/중앙_{start_date}_{end_date}_본문없음_최종_전체기업_크롤링.csv')

hanK_all = sample_filtered_all[sample_filtered_all['언론사']=='한국경제']
hanK_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/한국경제_{start_date}_{end_date}_본문없음_최종_전체기업_크롤링.csv')

hankyre_all = sample_filtered_all[sample_filtered_all['언론사']=='한겨레']
hankyre_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/한겨레_{start_date}_{end_date}_본문없음_최종_전체기업_크롤링.csv')

In [ ]:
len(yna_all), len(donga_all), len(joong_all), len(hanK_all), len(hankyre_all)

(154, 21, 27, 71, 9)

# 언론사별 크롤링...
- 연합뉴스, 한국경제, 동아일보, 조선일보, 중앙일보, 한겨레
- (데이터프레임 df: 기업/언론사/제목/일자/URL/본문)

### 동아일보(완료)

In [ ]:
def donga_extract_news(url):
  req = requests.get(url)
  soup = BeautifulSoup(req.text, 'html.parser')

  # 게시글 제목
  article_title = soup.find('title')
  article_title = article_title.get_text(strip=True) if article_title else "제목을 찾을 수 없습니다."


  # 게시글 본문
  article_body = soup.find('section', class_='news_view')
  article_text = ''
  if article_body:
      # Loop through all direct children of the article_body
      for child in article_body.children:
          if child.name == 'p':  # Extract text from <p> tags
              article_text += child.get_text(strip=True) + ' '
          elif child.name == 'br':  # Handle <br> tags as line breaks
              article_text += '\n'
          elif isinstance(child, str):  # Extract text nodes directly within the div
              article_text += child.strip() + ' '
  article_text = article_text.strip()

  # 기사 날짜 추출
  date_element = soup.find('span', {'aria-hidden': 'true'})
  article_date = date_element.get_text(strip=True) if date_element else "날짜를 찾을 수 없습니다."

  # 데이터프레임 생성
  data = {
      '기사 제목': [article_title],
      '게시날짜': [article_date],
      '본문': [article_text]
      }

  return pd.DataFrame(data)

In [ ]:
def donga_extract_info(df):
    # Filtering rows where '언론사' is '동아일보'
    donga_rows = df[df['언론사'] == '동아일보']

    # Iterating through filtered rows
    for index, row in donga_rows.iterrows():
        # Extract information using donga_extract_info function
        donga_info = donga_extract_news(row['URL'])

        # Merging with original row
        for column in donga_info.columns:
            df.at[index, column] = donga_info.at[0, column]

    return df

In [ ]:
sample_filtered_all = donga_extract_info(sample_filtered_all)

In [ ]:
#sample_filtered_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/동아_{start_date}_{end_date}_최종_전체기업_크롤링_본문.csv')

### 중앙일보(완료)

In [ ]:
def joongang_extract_news(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    # 기사 제목 추출
    article_title = soup.find('h1', class_='headline').get_text(strip=True)

    # 기사 본문 추출
    article_body = soup.find('div', class_='article_body fs3').get_text(strip=True)

    # 기사 날짜 추출
    time_published_element = soup.find('time', {'itemprop': 'datePublished'})
    posted_date = time_published_element['datetime'] if time_published_element else None

    time_modified_element = soup.find('time', {'itemprop': 'dateModified'})
    update_date = time_modified_element['datetime'] if time_modified_element else None

    if posted_date:
        posted_date = datetime.strptime(posted_date, "%Y-%m-%dT%H:%M:%S%z")
        posted_date = posted_date.strftime("%Y-%m-%d %H:%M:%S")

    if update_date:
        update_date = datetime.strptime(update_date, "%Y-%m-%dT%H:%M:%S%z")
        update_date = update_date.strftime("%Y-%m-%d %H:%M:%S")

    # 데이터프레임 생성
    data = {
        '기사 제목': [article_title],
        '게시날짜': [posted_date],
        '본문': [article_body]
    }
    return pd.DataFrame(data)

In [ ]:
# @title
def joongang_extract_info(df):
  joongang_rows = df[df['언론사'] == '중앙일보']

  # Iterating through filtered rows
  for index, row in joongang_rows.iterrows():
  # Extract information using donga_extract_info function
    joongang_info = joongang_extract_news(row['URL'])

    for column in joongang_info.columns:
      df.at[index, column] = joongang_info.at[0, column]

  return df

In [ ]:
sample_filtered_all = joongang_extract_info(sample_filtered_all)

In [ ]:
#sample_filtered_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/동아_{start_date}_{end_date}_중앙일보_동아일보_최종_전체기업_크롤링_본문.csv')

### 연합뉴스

숨긴 코드는 driver 초기화

In [ ]:
# @title
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from time import sleep
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re
import requests
import os
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

# Dataframe
import pandas as pd

# 2023.08.02 셀레니움 라이브러리 수정사항 반영
# https://stackoverflow.com/questions/76428561/typeerror-webdriver-init-got-multiple-values-for-argument-options
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service, options=options)

숨긴 코드는 전체 함수

In [ ]:
# @title
def yna_extract_news(url):
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')

        # 기사 제목 추출
        title_element = soup.find('h2', class_='tit')
        article_title = title_element.get_text(strip=True) if title_element else '제목 없음'

        # 기사 본문 추출
        article_body_elements = soup.find_all('p')
        article_body = "\n".join([element.get_text(strip=True) for element in article_body_elements])

        # 기사 날짜 추출
        time_published_element = soup.find('p', id='newsUpdateTime01')
        posted_date = time_published_element['data-published-time'] if time_published_element else None

        if posted_date:
            try:
                posted_date = datetime.strptime(posted_date, "%Y%m%d%H%M")
                posted_date = posted_date.strftime("%Y-%m-%d %H:%M:%S")
            except ValueError:
                posted_date = None

        # 데이터프레임 생성
        data = {
            '기사 제목': [article_title],
            '게시날짜': [posted_date],
            '본문': [article_body]
        }
        print(data)
        return pd.DataFrame(data)

In [ ]:
# @title
def yna_extract_info(df):
    yna_rows = df[df['언론사'] == '연합뉴스']

    # Iterating through filtered rows
    for index, row in yna_rows.iterrows():
        # Extract information using yna_extract_news function
        yna_info = yna_extract_news(row['URL'])

        for column in yna_info.columns:
                df.at[index, column] = yna_info.at[0, column]
        else:
            print(f"Failed to extract data for URL: {row['URL']}")

    return df

In [ ]:
from selenium.common.exceptions import NoSuchElementException

sample_filtered_all = yna_extract_info(sample_filtered_all)

In [ ]:
#sample_filtered_all[sample_filtered_all['언론사']=='연합뉴스']

,기업,언론사,제목,일자,URL,업종,규모,기사 제목,게시날짜,본문
0,오리온,연합뉴스,"KBL 컵대회, 10월 5∼13일 제천서 개최…상무 포함 11개 팀 출전",2024-06-27,https://www.yna.co.kr/view/AKR20240627068200007,식·음료,중견기업,제목 없음,2024-06-27 10:42:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-27 10:42\n(군산=연...
2,효성화학,연합뉴스,"""올레핀 비중 큰 석유화학社 신용등급 줄하향…공급과잉""",2024-06-27,https://www.yna.co.kr/view/AKR20240627102300008,에너지·화학,대기업,"나신평, 한화토탈·효성화학 등 하향…""당분간 어려움 계속될 것""",2024-06-27 13:46:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-27 13:46\n[생성AI...
3,삼성전자,연합뉴스,"코람코라이프인프라리츠, 강남역 DF타워 우선주 투자로 배당 강화",2024-06-27,https://www.yna.co.kr/view/AKR20240627038200008,전기·전자,대기업,제목 없음,2024-06-27 09:15:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-27 09:15\n코람코라이...
5,아성다이소,연합뉴스,"""온라인 주문하고 찾아가세요""…다이소, 픽업 서비스 매장 확대",2024-06-27,https://www.yna.co.kr/view/AKR20240627067400030,고객 서비스·리테일,중견기업,제목 없음,2024-06-27 10:40:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-27 10:40\n(서울=연...
8,삼성바이오로직스,연합뉴스,바이오특화단지 5곳 지정…2040년까지 36.3조원 민간투자,2024-06-27,https://www.yna.co.kr/view/AKR20240627116100003,의료·제약·바이오,대기업,인천·경기 '세계 1위 바이오 메가 클러스터'…대전 '혁신 신약 R&D 거점' 등,2024-06-27 14:49:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-27 14:49\n[삼성전자...
...,...,...,...,...,...,...,...,...,...,...
279,SK,연합뉴스,"SK온, 엑손모빌과 리튬 공급 MOU…""미국산 리튬 최대10만t 확보""",2024-06-26,https://www.yna.co.kr/view/AKR20240626015200003,솔루션·SI·CRM·ERP,대기업,"공급 시기·물량은 본계약 체결 후 확정…""IRA 충족 핵심 광물 확보 노력""",2024-06-26 07:22:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 07:22\n(서울=연...
280,SK,연합뉴스,"하나증권 ""SKT, 그룹 지배구조 개편 기대감에 주가 상승 예상""",2024-06-26,https://www.yna.co.kr/view/AKR20240626026000008,솔루션·SI·CRM·ERP,대기업,제목 없음,2024-06-26 08:26:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 08:26\n하나증권은...
281,SK,연합뉴스,"SKT, 37만원대 5G 스마트폰 '갤럭시 와이드7' 단독 출시",2024-06-26,https://www.yna.co.kr/view/AKR20240626024900017,솔루션·SI·CRM·ERP,대기업,제목 없음,2024-06-26 08:42:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 08:42\n[SK텔레...
282,SK,연합뉴스,"""mRNA 백신으로 팬데믹 대비""…플랫폼 개발 속도 내는 제약사들",2024-06-26,https://www.yna.co.kr/view/AKR20240625135800017,솔루션·SI·CRM·ERP,대기업,제목 없음,2024-06-26 09:00:00,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 09:00\n(서울=연...


In [ ]:
#sample_filtered_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/{start_date}_{end_date}_연합뉴스_중앙일보_동아일보_최종_전체기업_크롤링_본문.csv')

### 한국경제

밑에 숨긴 코드는 전체 함수

In [ ]:
# @title
def hanK_extract_news(url):
    try:
        req = requests.get(url)
        req.raise_for_status()
        soup = BeautifulSoup(req.text, 'html.parser')

        # 기사 제목 추출
        title_element = soup.find('h1', class_='headline')
        article_title = title_element.get_text(strip=True) if title_element else '제목 없음'

        # 기사 본문 추출
        body_element = soup.find('p', class_='lead')
        article_body = body_element.get_text(strip=True) if body_element else '본문 없음'

        # 기사 날짜 추출
        article_date = '날짜 없음'
        date_elements = soup.find_all('span', class_='item')
        for element in date_elements:
            if '입력' in element.text:
                date_element = element.find('span', class_='txt-date')
                if date_element:
                    article_date = date_element.get_text(strip=True)
                    break

        # 데이터프레임 생성
        data = {
            '기사 제목': [article_title],
            '게시날짜': [article_date],
            '본문': [article_body]
        }
        return pd.DataFrame(data)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

In [ ]:
# @title
def hanK_extract_info(df):
    hanK_rows = df[df['언론사'] == '한국경제']

    for index, row in hanK_rows.iterrows():
        if pd.notna(df.at[index, '기사 제목']) and pd.notna(df.at[index, '게시날짜']) and pd.notna(df.at[index, '본문']):
            continue  # 이미 데이터가 있으면 건너뜀

        hanK_info = hanK_extract_news(row['URL'])

        if hanK_info is not None:
            for column in hanK_info.columns:
                df.at[index, column] = hanK_info.at[0, column]
        else:
            print(f"Failed to extract data for URL: {row['URL']}")

    return df

In [ ]:
sample_filtered_all = hanK_extract_info(sample_filtered_all)

In [ ]:
#sample_filtered_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/{start_date}_{end_date}_한국경제_연합뉴스_중앙일보_동아일보_최종_전체기업_크롤링_본문.csv')

### 한겨레

아래는 모든 함수 집합

In [ ]:
# @title
def hankyre_extract_news(url):
    driver.get(url)

    # 기사 제목 추출
    title_xpath = '//*[@id="renewal2023"]/div[1]/div/div/article/h3'  # 수정된 XPath
    try:
        title_element = driver.find_element(By.XPATH, title_xpath)
        title = title_element.text
        print(f'    {title}')
    except NoSuchElementException:
        print(f"Title not found for URL: {url}")
        return None

    # 날짜 추출
    date_xpath = '//li[@class="ArticleDetailView_dateListItem__mRc3d"]/span'
    try:
        date_element = driver.find_element(By.XPATH, date_xpath)
        date_text = date_element.text
        article_date = datetime.strptime(date_text, '%Y-%m-%d %H:%M').date()  # 날짜 형식 변환
        print(f'    {article_date}')
    except NoSuchElementException:
        print(f"Date not found for URL: {url}")
        return None

    # 본문 추출
    passage_xpath = '//*[@id="renewal2023"]/div[3]'
    try:
        passage = driver.find_element(By.XPATH, passage_xpath).text
        print(f'    {passage}')
    except NoSuchElementException:
        print(f"Content not found for URL: {url}")
        return None

    # 데이터프레임 생성
    data = {
        '기사 제목': [title],
        '게시날짜': [article_date],
        '본문': [passage]
    }
    print(f'기사 제목: {title}, 게시날짜: {article_date}, 본문: {passage}')
    return pd.DataFrame(data)

def hankyre_extract_info(df):
    for index, row in df.iterrows():
        if row['언론사'] == '한겨레':
            hankyre_info = hankyre_extract_news(row['URL'])

            # Check if the extraction was successful
            if hankyre_info is not None:
                for column in hankyre_info.columns:
                    df.at[index, column] = hankyre_info.at[0, column]
            else:
                print(f"Failed to extract data for URL: {row['URL']}")

    return df

In [ ]:
# @title

def hankyre_extract_info(source_df):#, target_df):
    content = []
    # 각 페이지의 뉴스 기사 순회 (본문 크롤링)
    for i in range(len(source_df)):
        url = str(source_df['URL'].iloc[i])
        driver.get(url)
        time.sleep(0.01)
        title_element = str(source_df['제목'].iloc[i])
        title = title_element
        #//*[@id="renewal2023"]/div[2]/ul/li[2]/span
        date_xpath = '//*[@id="renewal2023"]/div[2]/ul/li[2]/span'
        try:
          date_element = driver.find_element(By.XPATH, date_xpath)
          date_text = date_element.text
          article_date = datetime.strptime(date_text, '%Y-%m-%d %H:%M')#.date()  # 날짜 형식 변환
        except NoSuchElementException:
          continue
        passage_xpath = '//*[@id="renewal2023"]/div[3]'
        try:
          passage = driver.find_element(By.XPATH, passage_xpath).text
        except NoSuchElementException:
          continue

        source_df['기사 제목'].iloc[i] = title
        source_df['게시날짜'].iloc[i] = article_date
        source_df['본문'].iloc[i] = passage

        #content.append([title, article_date, passage])
        print(i,"번째 기사 크롤링... ")
        print("제목: ",title, ", 일자:", article_date)

    # 데이터프레임 생성
    return source_df

In [ ]:
hankyre_all = hankyre_extract_info(hankyre_all)

In [ ]:
# '제목' 열을 기준으로 데이터를 병합하지 않고 값 복사
for i, sample_row in sample_filtered_all.iterrows():
    for j, hankyre_row in hankyre_all.iterrows():
        if sample_row['제목'] == hankyre_row['제목']:
            sample_filtered_all.at[i, '기사 제목'] = hankyre_row['기사 제목']
            sample_filtered_all.at[i, '게시날짜'] = hankyre_row['게시날짜']
            sample_filtered_all.at[i, '본문'] = hankyre_row['본문']

sample_filtered_all = sample_filtered_all.drop(columns=['기사 제목'])
sample_filtered_all['일자'] = sample_filtered_all['게시날짜']
sample_filtered_all = sample_filtered_all.drop(columns=['게시날짜'])

sample_filtered_all['일자'] = sample_filtered_all['일자'].astype(str).str.replace('.', '-')
sample_filtered_all['일자'] = sample_filtered_all['일자'].str[:16]
sample_filtered_all['일자'] = pd.to_datetime(sample_filtered_all['일자'], format='%Y-%m-%d %H:%M')
# '일자' 열을 기준으로 오름차순 정렬
sample_filtered_all = sample_filtered_all.sort_values(by='일자')
sample_filtered_all = sample_filtered_all.reset_index(drop=True)

숨긴건 원본 코드 다시 꺼내지 말기

In [ ]:
sample_filtered_all

,기업,언론사,제목,일자,URL,업종,규모,본문
0,한화,연합뉴스,"LG 켈리, 1안타 완봉 역투…한화 와이스, 데뷔전 6이닝 무실점(종합)",2024-06-26 00:11:00,https://www.yna.co.kr/view/AKR20240626000200007,에너지·화학,대기업,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 00:11\nLG 트윈...
1,한화,중앙일보,무료 땅콩만 매달 6t…韓 전 지점 '글로벌 톱10' 접수한 이곳,2024-06-26 01:06:00,https://www.joongang.co.kr/article/25259009,에너지·화학,대기업,파이브가이즈 4개 점포 전경. 사진 파이브가이즈미국 수제 햄버거 브랜드 파이브가이즈...
2,대상,연합뉴스,"러, 서방언론 웹사이트 81곳 접근차단…EU 반발",2024-06-26 01:21:00,https://www.yna.co.kr/view/AKR20240626001200088,식·음료,중견기업,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 01:21\n러시아 외...
3,삼성전자,연합뉴스,"삼성전자 DS부문, 오늘 글로벌전략회의…반도체 돌파구 모색 총력",2024-06-26 05:00:00,https://www.yna.co.kr/view/AKR20240625156700003,전기·전자,대기업,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 05:00\n[삼성전자...
4,삼성전자,중앙일보,"위기의 삼성 반도체, 오늘 전략회의 ""필수인원만, 빠르게 처리""",2024-06-26 05:00:00,https://www.joongang.co.kr/article/25259027,전기·전자,대기업,삼성전자 반도체가 생산되는 경기 화성캠퍼스 전경. 사진 삼성전자삼성전자가 하반기 반...
...,...,...,...,...,...,...,...,...
277,교보증권,연합뉴스,"국고채 금리, 美채권시장 약세에 상승…3년물 3.220%(종합)",2024-06-27 17:36:00,https://www.yna.co.kr/view/AKR20240627149651008,금융,대기업,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-27 17:36\n[강민지 ...
278,CJ+ENM,한국경제,"야구 이어 프로농구도 티빙으로…CJ ENM, KBL 4시즌 중계권 확보",2024-06-27 17:47:00,https://www.hankyung.com/article/2024062769557,NaN,NaN,"""지금 잘되는 거? 다음이 없잖아요.""티빙의 올해 기세가 심상치 않다. '환승연애3..."
279,한국주택금융공사,한겨레,"[사설] 국민 돈으로 부실PF 틀어막기, 미루면 다 해결되나",2024-06-27 18:00:00,https://www.hani.co.kr/arti/opinion/editorial/...,공공·복지,공공기관,기사를 읽어드립니다\n2:47\n대구 달서구에는 준공 후 2개월이 넘도록 분양이 이...
280,롯데건설,동아일보,"산업정책연구원, ‘2024 국가서비스대상’ 수상식 개최",2024-06-27 18:14:00,https://www.donga.com/news/Economy/article/all...,건설·토목,대기업,‘2024 국가서비스대상’ 수상식이 27일 서울 서초구 엘타워 그레이스홀에서 개최됐...


In [ ]:
sample_filtered_all.to_csv(f'/content/drive/MyDrive/뉴스레터 서비스(공유)/{start_date}_{end_date}_한겨레_한국경제_연합뉴스_중앙일보_동아일보_최종_전체기업_크롤링_본문.csv')

# 주요 언론사 모음 (최종 정리)

In [ ]:
df = sample_filtered_all.copy()
# Assuming you want to save 'sample_filtered' DataFrame to a CSV file
path = f'/content/drive/MyDrive/뉴스레터 서비스(공유)/{start_date}_{end_date}_기업데이터_요약_전.csv'
df = df.dropna(subset=['본문'])
new_order = ['기업', '일자', '제목', 'URL', '본문', '업종', '규모']#, '언론사']
df = df.reindex(columns=new_order)
#df.rename(columns={'본문': '최종 본문'}, inplace=True)
#df.to_csv(path, index=False)

,기업,언론사,제목,일자,URL,업종,규모,본문
0,한화,연합뉴스,"LG 켈리, 1안타 완봉 역투…한화 와이스, 데뷔전 6이닝 무실점(종합)",2024-06-26 00:11:00,https://www.yna.co.kr/view/AKR20240626000200007,에너지·화학,대기업,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 00:11\nLG 트윈...
1,한화,중앙일보,무료 땅콩만 매달 6t…韓 전 지점 '글로벌 톱10' 접수한 이곳,2024-06-26 01:06:00,https://www.joongang.co.kr/article/25259009,에너지·화학,대기업,파이브가이즈 4개 점포 전경. 사진 파이브가이즈미국 수제 햄버거 브랜드 파이브가이즈...
2,대상,연합뉴스,"러, 서방언론 웹사이트 81곳 접근차단…EU 반발",2024-06-26 01:21:00,https://www.yna.co.kr/view/AKR20240626001200088,식·음료,중견기업,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 01:21\n러시아 외...
3,삼성전자,연합뉴스,"삼성전자 DS부문, 오늘 글로벌전략회의…반도체 돌파구 모색 총력",2024-06-26 05:00:00,https://www.yna.co.kr/view/AKR20240625156700003,전기·전자,대기업,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-26 05:00\n[삼성전자...
4,삼성전자,중앙일보,"위기의 삼성 반도체, 오늘 전략회의 ""필수인원만, 빠르게 처리""",2024-06-26 05:00:00,https://www.joongang.co.kr/article/25259027,전기·전자,대기업,삼성전자 반도체가 생산되는 경기 화성캠퍼스 전경. 사진 삼성전자삼성전자가 하반기 반...
...,...,...,...,...,...,...,...,...
277,교보증권,연합뉴스,"국고채 금리, 美채권시장 약세에 상승…3년물 3.220%(종합)",2024-06-27 17:36:00,https://www.yna.co.kr/view/AKR20240627149651008,금융,대기업,자동완성 기능이 켜져 있습니다.\n송고시간2024-06-27 17:36\n[강민지 ...
278,CJ+ENM,한국경제,"야구 이어 프로농구도 티빙으로…CJ ENM, KBL 4시즌 중계권 확보",2024-06-27 17:47:00,https://www.hankyung.com/article/2024062769557,NaN,NaN,"""지금 잘되는 거? 다음이 없잖아요.""티빙의 올해 기세가 심상치 않다. '환승연애3..."
279,한국주택금융공사,한겨레,"[사설] 국민 돈으로 부실PF 틀어막기, 미루면 다 해결되나",2024-06-27 18:00:00,https://www.hani.co.kr/arti/opinion/editorial/...,공공·복지,공공기관,기사를 읽어드립니다\n2:47\n대구 달서구에는 준공 후 2개월이 넘도록 분양이 이...
280,롯데건설,동아일보,"산업정책연구원, ‘2024 국가서비스대상’ 수상식 개최",2024-06-27 18:14:00,https://www.donga.com/news/Economy/article/all...,건설·토목,대기업,‘2024 국가서비스대상’ 수상식이 27일 서울 서초구 엘타워 그레이스홀에서 개최됐...


In [ ]:
import json

path = '/content/drive/MyDrive/요약x_300대기업_data_언론사_제거_' + '_ds=' + start_date + '_de=' + end_date + "_"
df.to_csv(f'{path}.csv', index=False)

# 데이터프레임을 JSON 파일로 저장
pth = '/content/drive/MyDrive/json_요약x_300대기업_data_' + '_ds=' + start_date + '_de=' + end_date + "_"
df.to_json(f'{pth}.json', orient='records')

# 아 요약 시작하자

## 본문 요약1

In [ ]:
# python 3
!pip install newspaper3k
!pip install gensim sumy nltk

In [ ]:
sam = df.copy()

In [ ]:
# @title
import pandas as pd
import nltk
nltk.download('punkt')
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [ ]:
# Define the summarization function
def summarize_text(text):
    if pd.isna(text):
        return text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, 3)
    summarized_text = " ".join(str(sentence) for sentence in summary)
    return summarized_text

# Define a function to add summarized text to the DataFrame
def add_summary_column(df, text_column_name, summary_column_name):
    summary_list = []
    for text in df[text_column_name]:
        summarized_text = summarize_text(text)
        summary_list.append(summarized_text)
    df[summary_column_name] = summary_list
    return df

# Call the function to add a summary column
sam = add_summary_column(sam, '본문', '본문 요약1')
sam

## 본문 요약2

In [ ]:
# TextRank 모델 만들기

!pip install jpype1
!pip install konlpy
!pip install scikit-learn

In [ ]:
df = sam.copy()

In [ ]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [ ]:
# 자연어 처리
# 꼬꼬마 분석기...
# text를 문장 단위로 나누고 리스트로 저장 (문자열 길이 <= 10 ...... 이전 문장과 합치기)

# 꼬꼬만 분석기 불러오기
# TF-IDF 모델 생성 및 그래프 생성
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentences=[]
kkma = Kkma()

# text 입력 -> Kkma.sentences()를 이용하여 문장단위로 나누고 sentences로 리턴
def text2sentences(text):
  sentences = kkma.sentences(text)    # text일때 문장별로 리스트 만듬
  for idx in range(0,len(sentences)):
    if len(sentences[idx]) <= 10:
      sentences[idx-1] +=(' ' + sentences[idx])
      sentences[idx] = ' '
  return sentences

# 문장을 명사 형태로 변경
# Twitter.nouns() 사용하기
twitter = Twitter()
# 불용어제거
stopwords = ['연합뉴스', '동아일보', '중앙일보', '한겨레', '한국경제', '기자', '아', '휴', '우리', '저희', '을', '를', '은', '는','에','의', '가', '나']

def get_nouns(sentences):
  nouns=[]
  for sentence in sentences:
    if sentence is not '':
      nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence)) if noun not in stopwords and len(noun) > 1]))
  return nouns

#print(sentences[0])
#print()
#print(nouns[0])


# 명사로 이루어진 문장리스트 입력받음
# -> TfidVEctorizer.fit_transform 사용하여 TF-IDF matrix 만들기
# sent_graph 변수에 요약할 문장 단위에 대한 sentence graph 생성됨.
def build_sent_graph(sentence):
  tfidf_mat = tfidf.fit_transform(sentence).toarray()
  graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
  return graph_sentence

# 단어 단위에 대한 word graph 만들기
# {idx: word} 딕셔너리 형태 만들기 -> sklearn의 CountVectorizer 사용.
# 가장 순위가 높은 keyword 찾기
def build_words_graph(sentence):
  cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
  vocab = cnt_vec.vocabulary_
  return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

def get_ranks(graph, d=0.8):
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0
        link_sum = np.sum(A[:, id])
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1
    B = (1 - d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B)
    return {idx: r[0] for idx, r in enumerate(ranks)}

### 찐 TextRank 구현

In [ ]:
def summarize_2(text, sent_num=5):
    def text2sentences(text):
        # This function should split text into sentences
        return text.split('. ')

    def get_nouns(sentences):
        # This function should extract nouns from sentences for graph building
        return [sentence.split() for sentence in sentences]

    def build_sent_graph(nouns):
        # This function should build a graph based on the sentences
        return {i: len(noun) for i, noun in enumerate(nouns)}

    def get_ranks(sent_graph):
        # This function should rank the sentences
        return {idx: rank for idx, rank in sent_graph.items()}

    # Split the text into sentences
    sentences = text2sentences(text)

    # Extract nouns from sentences
    nouns = get_nouns(sentences)

    # Build a graph based on the sentences
    sent_graph = build_sent_graph(nouns)

    # Rank the sentences
    sent_rank_idx = get_ranks(sent_graph)

    # Sort the sentence ranks
    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)

    # Select the top ranked sentences
    index = sorted(sorted_sent_rank_idx[:sent_num])
    summary = [sentences[idx] for idx in index]

    return ' '.join(summary)

In [ ]:
content = []
df['본문 요약2'] = ""

for p in range(len(df)):
  article = df['본문'].iloc[p]

  # 기사 본문 내용 요약
  newsSum = summarize_2(article)
  df.at[p,'본문 요약2'] = newsSum

df.head(1)

,언론사,제목,게시일,기사 URL,게시날짜,본문,본문 요약1,본문 요약2,본문 요약4
0,한겨레,경남 합천·전남 구례에 양수발전소 들어선다,2024-01-10,https://www.hani.co.kr/arti/society/environmen...,NaN,"지난달 1일, 충남 금산군민들이 금산종합체육관에서 열린 양수발전소 유치 결의대회에서...","지난달 1일, 충남 금산군민들이 금산종합체육관에서 열린 양수발전소 유치 결의대회에서...",송인걸 기자 igsong@hani.co.kr\n2035년 이전에 국내에 핵발전(원전...,


## 본문 요약3

- https://m.blog.naver.com/serendipity86/223240547116

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from collections import defaultdict
from konlpy.tag import Kkma
from konlpy.tag import Okt

nltk.download('punkt')
okt = Okt()

def korean_sent_tokenize(text):
  kkma = Kkma()
  sentences = kkma.sentences(text)
  return sentences

def summarize_3(text, num_of_sentences=3):
  words = okt.morphs(text)
  # 단어 빈도수
  freqTable = defaultdict(int)
  for word in words:
    word = word.lower()
    freqTable[word] += 1

  # 문장 중요도
  sentences= korean_sent_tokenize(text)
  sentenceValue = defaultdict(int)

  for sentence in sentences:
    for word, freq in freqTable.items():
      if word in sentence.lower():
        sentenceValue[sentence] = sentenceValue[sentence] / len(okt.morphs(sentence))

  # 문장 중요도 정렬
  sorted_sentenceValue = sorted(sentenceValue.items(), key=lambda x: x[1], reverse=True)

  # 상위 문장들로 요약문 생성
  summary=[]
  for i in range(min(num_of_sentences, len(sorted_sentenceValue))):
    summary.append(sorted_sentenceValue[i][0])

  return " ".join(summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from collections import defaultdict
from konlpy.tag import Kkma
from konlpy.tag import Okt

nltk.download('punkt')
okt = Okt()

def korean_sent_tokenize(text):
    kkma = Kkma()
    sentences = kkma.sentences(text)
    return sentences

def summarize_korean_text(text, num_of_sentences=3):
    words = okt.morphs(text)

    # Word frequency
    freqTable = defaultdict(int)
    for word in words:
        word = word.lower()
        freqTable[word] += 1

    # Sentence importance
    sentences = korean_sent_tokenize(text)
    sentenceValue = defaultdict(int)

    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                sentenceValue[sentence] += freq

    # Sort sentence importance
    sorted_sentenceValue = sorted(sentenceValue.items(), key=lambda x: x[1], reverse=True)

    # Generate summary from top sentences
    summary = []
    for i in range(min(num_of_sentences, len(sorted_sentenceValue))):
        summary.append(sorted_sentenceValue[i][0])

    return " ".join(summary)

# Add summarized text to DataFrame
def add_summary_column(df, text_column_name, summary_column_name, num_of_sentences=3):
    df[summary_column_name] = df[text_column_name].apply(lambda x: summarize_korean_text(x, num_of_sentences))
    return df

# Example usage:
# df = add_summary_column(df, '본문', '본문 요약3')

# Display the DataFrame with the summary column
# print(df)
df = add_summary_column(df, '본문', '본문 요약3', num_of_sentences=3)
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,언론사,제목,게시일,기사 URL,게시날짜,본문,본문 요약1,본문 요약2,본문 요약4,본문 요약3
0,한겨레,경남 합천·전남 구례에 양수발전소 들어선다,2024-01-10,https://www.hani.co.kr/arti/society/environmen...,NaN,"지난달 1일, 충남 금산군민들이 금산종합체육관에서 열린 양수발전소 유치 결의대회에서...","지난달 1일, 충남 금산군민들이 금산종합체육관에서 열린 양수발전소 유치 결의대회에서...",송인걸 기자 igsong@hani.co.kr\n2035년 이전에 국내에 핵발전(원전...,,"산업 부는 이들 2 곳 외에 적격기준을 통과한 경북 영양( 한 수원 ·1GW), 경..."
1,연합뉴스,"구례군 ""1조4천억원 규모 양수발전소 유치 환영""",2024-01-10,https://www.yna.co.kr/view/AKR20231228165000054,2023-12-28,장아름 기자\n기자 페이지\n구례 양수발전소 기원행사\n[구례군 제공]\n(구례=연...,장아름 기자 기자 페이지 구례 양수발전소 기원행사 [구례군 제공] (구례=연합뉴스)...,장아름 기자\n기자 페이지\n구례 양수발전소 기원행사\n[구례군 제공]\n(구례=연...,,장 아름 기자 기자 페이지 구례 양수발전소 기원행사 [ 구례군 제공] ( 구례= 연...
2,연합뉴스,"LS엠트론, 동서발전·성우 등과 바이오가스 플랜트 사업 협력",2024-01-10,https://www.yna.co.kr/view/AKR20231228032800003,2023-12-28,LS엠트론이 제안하는 미래 농업 'CFA'(Carbon Free Agricultur...,협약으로 각사가 보유한 자원을 바탕으로 바이오가스 플랜트 사업모델을 개발하고 추진해...,LS엠트론이 제안하는 미래 농업 'CFA'(Carbon Free Agricultur...,,LS 엠 트론이 제안하는 미래 농업 'CFA' (Carbon Free Agricul...
3,동아일보,"동서식품, ‘맥심커피배 입신최강전’ 개최… 프로바둑 9단 진검승부 펼친다",2024-01-09,https://www.donga.com/news/Economy/article/all...,입력 2023-12-20 15:14,동서식품은 20일 서울 성동구 한국기원에서 ‘제25회 맥심커피배 입신최강전’ 개막식...,동서식품은 20일 서울 성동구 한국기원에서 ‘제25회 맥심커피배 입신최강전’ 개막식...,동서식품은 20일 서울 성동구 한국기원에서 ‘제25회 맥심커피배 입신최강전’ 개막식...,,‘ 맥 심 커피 배 입신 최강 전’ 은 동서식품이 국내 바둑 발전을 위해 지난 25...
4,연합뉴스,"농가 에너지시설 지원…농식품부, 한국동서발전·대상과 협약",2024-01-08,https://www.yna.co.kr/view/AKR20231213039800030,2023-12-13,신선미 기자\n기자 페이지\n농림축산식품부 현판과 건물\n[농식품부 제공]\n(서울...,신선미 기자 기자 페이지 농림축산식품부 현판과 건물 [농식품부 제공] (서울=연합뉴...,신선미 기자\n기자 페이지\n농림축산식품부 현판과 건물\n[농식품부 제공]\n(서울...,,신선미 기자 기자 페이지 농림 축산식품 부 현판과 건물 [ 농식품 부 제공] ( 서...
5,한국경제,"HJ중공업, 발전소 해체공사로 새해 마수걸이 수주",2024-01-05,https://www.hankyung.com/article/202401042135i,2024-01-04,울산기력 전경사진. HJ중공업 제공\nHJ중공업 건설부문은 한국동서발전이 발주한 울...,운영 초기 국내 총 전력 생산량의 약 15%를 담당했다. 정부는 지난해 1월 발표한...,HJ중공업 건설부문의 올해 첫 수주 실적이다.\n\n1980년 가동한 울산기력 4·...,,"HJ 중공업 관계자는 “ 국내외 발전시설 시공 경험과 노하우로 울산 기력 4, 5,..."
6,연합뉴스,"HJ중공업, '국내 최대 중유발전' 울산기력 4∼6호기 해체 수주",2024-01-05,https://www.yna.co.kr/view/AKR20240104074400003,2024-01-04,권혜진 기자\n기자 페이지\n울산기력 전경사진\n[HJ중공업 제공. 재판매 및 DB...,"울산기력 4·5·6호기는 총 1천200㎿ 설비용량을 갖췄던 국내 최대 중유발전소로,...",권혜진 기자\n기자 페이지\n울산기력 전경사진\n[HJ중공업 제공 재판매 및 DB ...,,"HJ 중공업 관계자는 "" 국내외 발전시설 시공 경험과 노하우로 울산 기력 4∼6 호..."
7,한국경제,"HJ중공업, 울산 기력발전 해체공사 575억원에 수주",2024-01-05,https://www.hankyung.com/article/202401042247h,2024-01-04,HJ중공업 건설부문(대표 홍문기)은 한국동서발전이 발주한 울산기력 4~ 6호기 해체...,1980년부터 가동된 울산기력 발전소는 총 1200㎿의 설비 용량을 갖춘 국내 최대...,HJ중공업 건설부문(대표 홍문기)은 한국동서발전이 발주한 울산기력 4~ 6호기 해체...,,"HJ 중공업은 현재 보령 신복합 1 호기 발전소 및 신 세종 복합 발전소, 강릉 안..."
8,중앙일보,"동서발전, ‘2023년 안전문화대상’ 국무총리 표창 수상",2023-12-12,https://www.joongang.co.kr/article/25214121,2023-12-12 11:50:00,동서발전은 11일(월) 오후 3시 정부세종컨벤션센터에서 열린 '2023 안전문화대상...,동서발전은 11일(월) 오후 3시 정부세종컨벤션센터에서 열린 '2023 안전문화대상...,동서발전은 11일(월) 오후 3시 정부세종컨벤션센터에서 열린 '2023 안전문화대상...,,동서 발전은 ‘ 안전 최우선’ 경영방침을 토대로 매년 전직원이 참여하는 ‘ 다 함께...
9,중앙일보,"동서발전 김영문 사장, 올해 글로벌품질경영인대상 수상",2023-11-28,https://www.joongang.co.kr/article/25210724,2023-11-28 14:25:00,27(월) 오전 11시 동서발전 본사에서 '2023년 글로품질경영인대상'을 수상한 ...,27(월) 오전 11시 동서발전 본사에서 '2023년 글로품질경영인대상'을 수상한 ...,27(월) 오전 11시 동서발전 본사에서 '2023년 글로품질경영인대상'을 수상한 ...,,"글로 벌 품질 경영인대상은 우리나라 산업발전과 품질경영 활동에 크게 공헌하고, 기업..."


## 본문 요약4
- https://mumin-blog.tistory.com/207#google_vignette

In [ ]:
!pip install summa

In [ ]:
from summa.summarizer import summarize

def summarize_text_with_summa(text, num_of_sentences=3):
    return summarize(text, words=num_of_sentences)

# Add summarized text to DataFrame
def add_summary_column_with_summa(df, text_column_name, summary_column_name, num_of_sentences=3):
    df[summary_column_name] = df[text_column_name].apply(lambda x: summarize_text_with_summa(x, num_of_sentences))
    return df

# Example usage:
df = add_summary_column_with_summa(df, '본문', '본문 요약4')
df

,언론사,제목,게시일,기사 URL,게시날짜,본문,본문 요약1,본문 요약2,본문 요약4
0,한겨레,경남 합천·전남 구례에 양수발전소 들어선다,2024-01-10,https://www.hani.co.kr/arti/society/environmen...,NaN,"지난달 1일, 충남 금산군민들이 금산종합체육관에서 열린 양수발전소 유치 결의대회에서...","지난달 1일, 충남 금산군민들이 금산종합체육관에서 열린 양수발전소 유치 결의대회에서...","산업 부는 이들 2 곳 외에 적격기준을 통과한 경북 영양( 한 수원 ·1GW), 경...",
1,연합뉴스,"구례군 ""1조4천억원 규모 양수발전소 유치 환영""",2024-01-10,https://www.yna.co.kr/view/AKR20231228165000054,2023-12-28,장아름 기자\n기자 페이지\n구례 양수발전소 기원행사\n[구례군 제공]\n(구례=연...,장아름 기자 기자 페이지 구례 양수발전소 기원행사 [구례군 제공] (구례=연합뉴스)...,장 아름 기자 기자 페이지 구례 양수발전소 기원행사 [ 구례군 제공] ( 구례= 연...,
2,연합뉴스,"LS엠트론, 동서발전·성우 등과 바이오가스 플랜트 사업 협력",2024-01-10,https://www.yna.co.kr/view/AKR20231228032800003,2023-12-28,LS엠트론이 제안하는 미래 농업 'CFA'(Carbon Free Agricultur...,협약으로 각사가 보유한 자원을 바탕으로 바이오가스 플랜트 사업모델을 개발하고 추진해...,LS 엠 트론이 제안하는 미래 농업 'CFA' (Carbon Free Agricul...,
3,동아일보,"동서식품, ‘맥심커피배 입신최강전’ 개최… 프로바둑 9단 진검승부 펼친다",2024-01-09,https://www.donga.com/news/Economy/article/all...,입력 2023-12-20 15:14,동서식품은 20일 서울 성동구 한국기원에서 ‘제25회 맥심커피배 입신최강전’ 개막식...,동서식품은 20일 서울 성동구 한국기원에서 ‘제25회 맥심커피배 입신최강전’ 개막식...,‘ 맥 심 커피 배 입신 최강 전’ 은 동서식품이 국내 바둑 발전을 위해 지난 25...,
4,연합뉴스,"농가 에너지시설 지원…농식품부, 한국동서발전·대상과 협약",2024-01-08,https://www.yna.co.kr/view/AKR20231213039800030,2023-12-13,신선미 기자\n기자 페이지\n농림축산식품부 현판과 건물\n[농식품부 제공]\n(서울...,신선미 기자 기자 페이지 농림축산식품부 현판과 건물 [농식품부 제공] (서울=연합뉴...,신선미 기자 기자 페이지 농림 축산식품 부 현판과 건물 [ 농식품 부 제공] ( 서...,
5,한국경제,"HJ중공업, 발전소 해체공사로 새해 마수걸이 수주",2024-01-05,https://www.hankyung.com/article/202401042135i,2024-01-04,울산기력 전경사진. HJ중공업 제공\nHJ중공업 건설부문은 한국동서발전이 발주한 울...,운영 초기 국내 총 전력 생산량의 약 15%를 담당했다. 정부는 지난해 1월 발표한...,"HJ 중공업 관계자는 “ 국내외 발전시설 시공 경험과 노하우로 울산 기력 4, 5,...",
6,연합뉴스,"HJ중공업, '국내 최대 중유발전' 울산기력 4∼6호기 해체 수주",2024-01-05,https://www.yna.co.kr/view/AKR20240104074400003,2024-01-04,권혜진 기자\n기자 페이지\n울산기력 전경사진\n[HJ중공업 제공. 재판매 및 DB...,"울산기력 4·5·6호기는 총 1천200㎿ 설비용량을 갖췄던 국내 최대 중유발전소로,...","HJ 중공업 관계자는 "" 국내외 발전시설 시공 경험과 노하우로 울산 기력 4∼6 호...",
7,한국경제,"HJ중공업, 울산 기력발전 해체공사 575억원에 수주",2024-01-05,https://www.hankyung.com/article/202401042247h,2024-01-04,HJ중공업 건설부문(대표 홍문기)은 한국동서발전이 발주한 울산기력 4~ 6호기 해체...,1980년부터 가동된 울산기력 발전소는 총 1200㎿의 설비 용량을 갖춘 국내 최대...,"HJ 중공업은 현재 보령 신복합 1 호기 발전소 및 신 세종 복합 발전소, 강릉 안...",
8,중앙일보,"동서발전, ‘2023년 안전문화대상’ 국무총리 표창 수상",2023-12-12,https://www.joongang.co.kr/article/25214121,2023-12-12 11:50:00,동서발전은 11일(월) 오후 3시 정부세종컨벤션센터에서 열린 '2023 안전문화대상...,동서발전은 11일(월) 오후 3시 정부세종컨벤션센터에서 열린 '2023 안전문화대상...,동서 발전은 ‘ 안전 최우선’ 경영방침을 토대로 매년 전직원이 참여하는 ‘ 다 함께...,
9,중앙일보,"동서발전 김영문 사장, 올해 글로벌품질경영인대상 수상",2023-11-28,https://www.joongang.co.kr/article/25210724,2023-11-28 14:25:00,27(월) 오전 11시 동서발전 본사에서 '2023년 글로품질경영인대상'을 수상한 ...,27(월) 오전 11시 동서발전 본사에서 '2023년 글로품질경영인대상'을 수상한 ...,"글로 벌 품질 경영인대상은 우리나라 산업발전과 품질경영 활동에 크게 공헌하고, 기업...",


In [ ]:
final_df = df.copy()
final_df = final_df.drop('게시날짜', axis=1)

path = '/content/drive/MyDrive/요약o_300대기업_data' + '_ds=' + start_date + '_de=' + end_date + "_"
final_df.to_csv(f'{path}.csv', index=False)

pth = '/content/drive/MyDrive/json_요약o_300대기업_data' + '_ds=' + start_date + '_de=' + end_date + "_"
final_df.to_json(f'{pth}.json', orient='records')

In [ ]:
final_df